# 【課題】量子カーネルを使った新現象の分類

## 素粒子探索への応用

データセットは実習で使ったものと同じです。

In [1]:
# Tested with python 3.8.12, qiskit 0.34.2, numpy 1.22.2
# General imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# scikit-learn imports
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Qiskit imports
from qiskit import QuantumCircuit, Aer, transpile
from qiskit.circuit import QuantumCircuit, Parameter, ParameterVector
from qiskit.circuit.library import PauliFeatureMap, ZFeatureMap, ZZFeatureMap
from qiskit.circuit.library import TwoLocal, NLocal, RealAmplitudes, EfficientSU2
from qiskit.circuit.library import HGate, RXGate, RYGate, RZGate, CXGate, CRXGate, CRZGate
from qiskit_machine_learning.kernels import QuantumKernel

In [141]:
# ファイルから変数を読み出す
df = pd.read_csv("data/SUSY_1K.csv",
                 names=('isSignal','lep1_pt','lep1_eta','lep1_phi','lep2_pt','lep2_eta',
                        'lep2_phi','miss_ene','miss_phi','MET_rel','axial_MET','M_R','M_TR_2',
                        'R','MT2','S_R','M_Delta_R','dPhi_r_b','cos_theta_r1'))

# 学習に使う変数の数
feature_dim = 3  # dimension of each data point

# 3, 5, 7変数の場合に使う変数のセット
if feature_dim == 3:
    SelectedFeatures = ['lep1_pt', 'lep2_pt', 'miss_ene']
elif feature_dim == 5:
    SelectedFeatures = ['lep1_pt','lep2_pt','miss_ene','M_TR_2','M_Delta_R']
elif feature_dim == 7:
    SelectedFeatures = ['lep1_pt','lep1_eta','lep2_pt','lep2_eta','miss_ene','M_TR_2','M_Delta_R']

# 学習に使う事象数: trainingは訓練用サンプル、testingはテスト用サンプル
train_size = 20
test_size = 20

# オプティマイザーをCallする回数の上限
niter = 300
random_seed = 10598

df_sig = df.loc[df.isSignal==1, SelectedFeatures]
df_bkg = df.loc[df.isSignal==0, SelectedFeatures]

# サンプルの生成
df_sig_train = df_sig.values[:train_size]
df_bkg_train = df_bkg.values[:train_size]
df_sig_test = df_sig.values[train_size:train_size+test_size]
df_bkg_test = df_bkg.values[train_size:train_size+test_size]
train_data = np.concatenate([df_sig_train,df_bkg_train])
test_data = np.concatenate([df_sig_test,df_bkg_test])
#print('train_data =',training_data)
#print('test_data =',test_data)
train_label = np.concatenate([np.ones((train_size),dtype=int),np.zeros((train_size),dtype=int)])
test_label = np.concatenate([np.ones((test_size),dtype=int),np.zeros((test_size),dtype=int)])

train_label_one_hot = np.zeros((train_size*2, 2))
for i in range(train_size*2):
    train_label_one_hot[i, train_label[i]] = 1
test_label_one_hot = np.zeros((test_size*2, 2))
for i in range(test_size*2):
    test_label_one_hot[i, test_label[i]] = 1

mms = MinMaxScaler((-1, 1))
norm_train_data = mms.fit_transform(train_data)
norm_test_data = mms.transform(test_data)

### 問題1

各自特徴量マップを選び、実装してください。

In [142]:
##################
### EDIT BELOW ###
##################

feature_map = ...

##################
### EDIT ABOVE ###
##################

### 問題2

最初の二つのデータ$x_0=$norm_train_data[0]と$x_1=$norm_train_data[1]に対するカーネル行列要素を計算するための量子回路を作ってください。このページの下にQuantumKernelクラスを使った例がありますが、ここでは上で決めた特徴量マップに合うように自分で量子回路を書いてください。その回路に$x_0$と$x_1$を入力として与えた回路をqc_circuitとして作るという問題です。

In [5]:
##################
### EDIT BELOW ###
##################

qc_circuit = QuantumCircuit(...)

##################
### EDIT ABOVE ###
##################

作った量子回路をシミュレータで実行して、全ての量子ビットで0を測定する確率を計算してください。

In [ ]:
backend = Aer.get_backend('qasm_simulator')
qc_circuit = transpile(qc_circuit, backend=backend, seed_transpiler=random_seed)
job = backend.run(qc_circuit, shots=8192, seed_simulator=random_seed)

counts = job.result().get_counts(qc_circuit)
print(f"Probability of observing 0^n state = {counts['0'*feature_dim]/sum(counts.values())}")

Qiskitには特徴量マップから量子カーネルを作ってくれるAPI（QuantumKernelクラス）が準備されていて、それを使えば課題2は実は簡単に実行できます。

In [ ]:
q_kernel = QuantumKernel(feature_map=feature_map, quantum_instance=Aer.get_backend('statevector_simulator'))

qc_circuit = q_kernel.construct_circuit(norm_train_data[0], norm_train_data[1])
qc_circuit.decompose().decompose().draw('mpl')

QuantumKernelを使うと、カーネル行列を直接書き出して見ることも容易にできます。学習データから求めたカーネル行列と、学習データとテストデータから計算したカーネル行列をプロットしてみます。

In [ ]:
matrix_train = q_kernel.evaluate(x_vec=norm_train_data)
matrix_test = q_kernel.evaluate(x_vec=norm_test_data, y_vec=norm_train_data)

fig, axs = plt.subplots(1, 2, figsize=(10, 5))
axs[0].imshow(np.asmatrix(matrix_train), interpolation='nearest', origin='upper', cmap='Blues')
axs[0].set_title("training kernel matrix")
axs[1].imshow(np.asmatrix(matrix_test), interpolation='nearest', origin='upper', cmap='Reds')
axs[1].set_title("validation kernel matrix")
plt.show()

最後に、sklearnパッケージに実装されているサポートベクターマシンを使って分類を行います。量子回路学習の場合と同様に、データサイズや特徴量マップを変えるなどして分類精度がどう変わるか調べてみてください。

In [ ]:
qc_svc = SVC(kernel='precomputed')
qc_svc.fit(matrix_train, train_label)

train_score = qc_svc.score(matrix_train, train_label)
test_score = qc_svc.score(matrix_test, test_label)

print(f'Precomputed kernel: Classification Train score: {train_score*100}%')
print(f'Precomputed kernel: Classification Test score:  {test_score*100}%')

**提出するもの**

- 選んだ特徴量マップの説明とそのコード（問題１）
- カーネル行列要素を計算するための量子回路のコードと、その回路を使って計算した0を測定する確率の結果（問題２）
- 授業で行った変分量子回路を使った量子機械学習との比較（この課題の提出様式は問いません。ノートブックにMarkdownを追加して書き込んでもらってもいいですし、別ファイルとして提出してもらってもOKです）
   - 二つの方法を同じ条件（特徴量の変数、データサンプルのサイズ、特徴量マップ）で比較した時に、分類性能に対して何か系統的な違いは見えるでしょうか。特徴量やサンプルサイズを変えて比較するなどして、その振る舞いを自分なりに考察してみてください。
   - 一方が他方に比べて系統的に分類性能が悪くなっている場合、どうすれば悪い方を改善できるでしょうか。サンプルサイズが小さい時には、どちらの方法でも過学習（テストデータでの分類性能が訓練データでの分類性能より悪くなる）の傾向が見えていると思います。過学習をできるだけ抑えながら、分類性能を改善する方法がないか、考察してみてください。